# RFM ANALİZİ
- Bu ödevde bir marketin satış 2010-2011 satış verilerinin üzerinde RFM analizi uygulanmıştır.

**Değişkenler**
- InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder.
- StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
- Description: Ürün ismi
- Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
- InvoiceDate: Fatura tarihi ve zamanı
- UnitPrive: Ürün fiyatı (Sterlin Cinsinden)
- CustomerID: Eşsiz müşteri numarası
- Country: Ülke ismi. Müşterinin yaşadığı ülke.

In [ ]:
# GEREKLİ KÜTÜPHANELER TANIMLANIR...
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#to display all columns and rows:
pd.set_option('display.max_columns',None);
pd.set_option('display.max_rows',None);

# virgülden sonra gösterilecek olan basamak sayısı:
pd.set_option('display.float_format', lambda x:'%.2f' %x)

In [ ]:
# RFM ANALİZİ İÇİN VERİSETİ OKUTULUR...
df_2010_2011 = pd.read_excel("../input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx", sheet_name = "Year 2010-2011")

In [ ]:
# HERHANGİ BİR DURUMDA VERİSETİNİN ORJİNALİNE İHTİYAÇ DUYULMASI HALİNDE  VERİSETİ KOPYALANIP BİR YERDE SAKLANABİLİR...
df = df_2010_2011.copy()

In [ ]:
# VERİSETİNDE HANGİ GÖZLEM BİRİMLERİNE SAHİP OLDUĞUNA BAKILIR...
df.head()

In [ ]:
# VERİSETİNDEKİ VERİNİN TÜRLERİ VE DİĞER BİLGİLERE BAKILIR.
df.info()

In [ ]:
# VERİSETİNDE KAÇ ADET EKSİK VERİ OLDUĞUNA BAKILIR...
df.isnull().sum()

In [ ]:
# EKSİK VERİLER İŞLEM KOLAYLIĞI İÇİN KALICI OLARAK SİLİNİR...
df.dropna(inplace = True)
df.isnull().sum()

In [ ]:
# VERİSETİNDE CUSTOMER ID'LER FLOAT OLARAK GÖRÜNMEKTE. BUNLARI İNTEGER DÖNÜŞÜMÜ YAPILIR.
df["Customer ID"] = df["Customer ID"].astype(int)
df.head()

In [ ]:
# VERİSETİNDE AYKIRI VERİNİN OLUP OLMADIĞINA BAKILIR.
df.describe([0.01,0.05,0.10,0.25,0.50,0.75,0.90,0.95,0.99]).T

In [ ]:
# MAX DEĞERLERİ VE %99 DEĞERLERİ ARASINDA BÜYÜK FARKLAR OLDUĞU GÖRÜLMEKTEDİR. MAX DEĞER QUANTİTİY İÇİN 80995 İKEN %99 DEĞERİ 120'DİR. AYNI İŞLEM DİĞERLERİ İÇİNDE GEÇERLİDİR.
# KAÇ ADET AYKIRI GÖZLEM OLDUĞUNU BULUNUR.
for feature in ["Quantity","Price"]:
    Q1 = df[feature].quantile(0.01)
    Q3 = df[feature].quantile(0.99)
    IQR = Q3 - Q1
    upper = Q3 + 1.5 * IQR
    lower = Q1 - 1.5 * IQR
    
    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis = None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
    else:
        print(feature,"no")

In [ ]:
# İŞLEMLER SADECE SATIŞ YAPILAN ÜRÜNLER İÇİN HESAPLANMALIDIR. FATURA NUMALARININ BAŞINDA C OLANLAR İPTAL EDİLEN ÜRÜNLER OLDUĞU İÇİN BU ÜRÜNLER VERİSETİNDEN KALDIRILMALIDIR.
df = df[~df["Invoice"].str.contains("C", na=False)]

In [ ]:
# MONİTERY DEĞERİNİN HESAPLANABİLMESİ İÇİN TOTAL PRİCE DEĞERLERİNİN HESAPLANMASI GEREKMEKTEDİR.
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.head()

# RECENCY

In [ ]:
# TEMEL OLARAK MÜŞTERİNİN EN SON KAÇ GÜN ÖNCE ALIŞVERİŞ YAPTIĞI BULUNUR.
# SON FATURA TARİHİ REQUENCY'NİN HESAPLANMASI İÇİN KULLANILIR. SON FATURA TARİHİNİN BULUNUR...
df["InvoiceDate"].max()

In [ ]:
# SON FATURA TARİHİ BU GÜNÜN TARİHİ OLARAK KABUL EDİLER...
import datetime as dt
today_date = dt.datetime(2011,12,9)
today_date

In [ ]:
# SON FATURA TARİHİ TÜM TARİHLERDEN TEK TEK ÇIKARTILIR. ÇIKAN İŞLEM SONUÇLARI "... DAYS ..." OLDUĞU İÇİN İNTEGER DEĞERLERE DÖNÜŞTÜRÜLÜR. BUNUN İÇİN .DAYS KULLANILIR.
recency_df = df.groupby("Customer ID").agg({"InvoiceDate": lambda x: (today_date - x.max()).days})
recency_df.rename(columns = {"InvoiceDate":"Recency"}, inplace = True)
recency_df.head()

# FREQUENCY

In [ ]:
# TEMEL OLARAK MÜŞTERİNİN KAÇ ADET ALIŞVERİŞ YAPTIĞI BULUNUR.
# BU İŞLEM İÇİN CUSTOMER ID 'LERE GÖRE FATURALARIN EŞSİZ OLANLARI ALINIR. BÖYLECE HANGİ MÜŞTERİNİN KAÇ ADET ALIŞVERİŞ YAPTIĞI BULUNABİLİR.
freq_df = df.groupby(["Customer ID"]).agg({"InvoiceDate":"nunique"})
freq_df.rename(columns = {"InvoiceDate": "Frequency"}, inplace = True)
freq_df.head()

In [ ]:
# TEMEL OLARAK MÜŞTERİLERİN NE KADAR ÜCRET BIRAKTIĞI BULUNUR.
# BU İŞLEM İÇİN CUSTOMER'ID LERE GÖRE FATURALARIN TOPLAM TUTARINI ALINIR.
monetary_df = df.groupby("Customer ID").agg({"TotalPrice":"sum"})
monetary_df.rename(columns = {"TotalPrice": "Monetary"}, inplace = True)
monetary_df.head()

In [ ]:
# RFM puanlaması oluşturulur. Recency değeri için en küçük değer en büyük puan olduğu için bu işleme dikkat edilir.
rfm = pd.concat([recency_df,freq_df,monetary_df], axis=1)
rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels = [5, 4, 3, 2, 1])
rfm["FrequencyScore"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels = [1, 2, 3, 4, 5])
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"], 5, labels = [1, 2, 3, 4, 5])
rfm["RFM_Score"] = (rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)+ rfm['MonetaryScore'].astype(str))
rfm.head()

In [ ]:
# MÜŞTERİLERİN RFM PUANLARINA GÖRE SINIFLANDIRILMASI YAPILIR. SINIFLANDIRMA İÇİN R VE F DEĞERLERİ KULLANILIR MÜŞTERLERİN M DEĞERİNE İHTİYAÇ YOKTUR.
# REGEX KULLANILARAK İSİMLENDİRİLME YAPILIR.
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2][5]'  : 'Cant Loose',
    r'3[1-2]'    : 'About to Sleep',
    r'33'        : 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41'        : 'Promising',
    r'51'        : 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]'    : 'Champions'    
}

rfm['Segment']=(rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str))
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

In [ ]:
rfm[["Segment","Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count","median"])

In [ ]:
new_df = pd.DataFrame()
new_df["Loyal CustomersID"] = rfm[rfm["Segment"] == "Loyal Customers"].index
new_df.to_csv("Loyal_Customers.csv")

# YORUMLAR
## Şampiyonlar İçin Yorumlar
Şampiyonlar bu işletmenin gelirlerinin %49'a yakınını oluşturuyor. Bu nedenle bu kişileri kaybetmememiz gerekiyor. Fakat bu kişiler zaten bizden sık ve çok alışveriş yapıyorlar. Bu kişilere çok indirim yapmak, kazandırdıkları parayı düşürebilir. Bir insan olarak sık alışveriş yaptığımız yerden diğerlerine göre ayrıcalık bekleriz. O halde bu işletme :

- 1-Şampiyonlara özel üyelik sistemi getirmeli. Üyelik sisteminde %3-7 belki %10'lara varan indirimler yapılabilir. Bu indirimler sürekli olmamalı ve genel indirim zamanlarına da denk gelmemeli. Belki süreli indirim olabilir. Her ay başlangıcından itibaren 1 hafta gibi bir süre bu indirimlerden yararlanabilirler.

- 2-Bu kişilere zaman zaman güzel dileklerde bulunmak, önemli hissettirecek mesajlar atmak değerli hissettirecektir ve bu da kalıcılığı arttıracaktır. Her insan, önemli hissetmeyi sever.

- 3-Promosyon linkleri olmalı ve bunlar da ödüllendirilmeli. Yeni bir müşteri getirme başına bir hediye sistemi olabilir. Bu ayrıca yeni müşteri de çekecektir.

## Aksiyon Gerekli İçin Yorumlar
Bu kişiler işletmenin gelirlerinin yaklaşık %2'sini oluşturmakta. Bu kişiler bizden yeterince alışveriş yapıyorlar fakat arzu ettiğimiz geliri bize kazandırmıyorlar.

- 1- Bu kişilerin alışveriş faturaları incelenmeli, apriori gibi algoritmalarla analiz edilmeliler. Bu şekilde fazla ürün almaları sağlanmaya çalışılabilir.

- 2- Zaman zaman hatırlatıcı mesajlar atılmalı fakat sık olursa bu kişide hoşnutsuzluğa sebep olabilir. Bu yüzden ayda 1 veya 2 mesaj atılabilir. Bu mesajlar iyi dilekler içermeli belki de birkaç gün geçerli %10-15 indirim kuponu sağlanmalı ya da 3 al 2 öde şeklinde kampanyaların olduğu haber edilmeli.

## Yeni Müşteriler İçin Yorumlar
Yeni müşteriler bir işletmede kan tazeletecektir. Yeni ürünlerin satılması imkanı sağlayabilir, yeni pazar payları elde etmeye araç olabilirler. Her insan yeni bir yere girdiğinde bir gariplik hisseder ve araştırmak, güvende olma isteği duyar. Bu yüzden bu kişilere :

- 1- Güvenlice alışveriş yapabileceklerine dair 1-2 bilgilendirici mesaj gönderilebilir.

- 2- Yeni gelenlere özel küçük promosyonlar yapılabilir. X ürünü alana Y ürünü %10 indirimli gibi kampanyalar yapılabilir.

- 3- İlk 3 alışverişte %5-10 arasında indirim uygulanabilir.

- 4- Çok fazla reklam göstermek müşteride negatif etki yapacaktır. Belli başlı reklamlar veya haberler müşterinin ilgisini çekebilir.